In [ ]:
import shapely.geometry
import time, requests
import json

from IPython.display import display
from ipywidgets import *
from ipyleaflet import *
from ipydatetime import DatetimePicker
from osprey_flask_app import create_app

In [ ]:
f = open("domains.json")
data = json.load(f)

peace_coords = [
    (coord[0], coord[1]) for coord in data["borders"]["peace"]["coordinates"]
]
fraser_coords = [
    (coord[0], coord[1]) for coord in data["borders"]["fraser"]["coordinates"]
]
columbia_coords = [
    (coord[0], coord[1]) for coord in data["borders"]["columbia"]["coordinates"]
]

In [ ]:
def handle_click(**kwargs):
    if kwargs.get("type") == "click":
        lat = round(kwargs.get("coordinates")[0], 5)
        lon = round(kwargs.get("coordinates")[1], 5)

        if in_polygon(lat, lon):
            clear_previous_marker()

            points.append(f"{lat}, {lon}")

            point_list.options = points

            latitude.value = f"{lat}"
            longitude.value = f"{lon}"

            m.add_layer(Marker(location=kwargs.get("coordinates"), name="Marker"))


def handle_button(arg):
    print("The button has been clicked")

In [ ]:
def clear_previous_marker():
    for i in range(1, len(m.layers)):
        if m.layers[i].name == "Marker":
            m.remove_layer(m.layers[i])


def in_polygon(lat, lon):

    point = shapely.geometry.Point(lat, lon)

    peace_polygon = shapely.geometry.polygon.Polygon(p.locations)
    fraser_polygon = shapely.geometry.polygon.Polygon(f.locations)
    columbia_polygon = shapely.geometry.polygon.Polygon(c.locations)

    if (
        peace_polygon.contains(point)
        or fraser_polygon.contains(point)
        or columbia_polygon.contains(point)
    ):
        return True

    return False


def date_widget(descr):
    return DatetimePicker(description=descr, disabled=False)

In [ ]:
mapnik = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
mapnik.base = True
mapnik.name = "Default"

satellite = basemap_to_tiles(basemaps.Gaode.Satellite)
satellite.base = True
satellite.name = "Satellite"

m = Map(
    basemap=mapnik,
    center=(50.5, -120),
    zoom=5,
    layout=Layout(width="50%", height="800px"),
    layers=[satellite, mapnik],
)

p = Polygon(
    locations=peace_coords,
    color="blue",
    name="Peace",
)
f = Polygon(
    locations=fraser_coords,
    color="red",
    name="Fraser",
)
c = Polygon(
    locations=columbia_coords,
    color="green",
    name="Columbia",
)

legend = LegendControl(
    {"Peace": "blue", "Fraser": "red", "Columbia": "green"},
    name="Watersheds",
    position="topright",
)

latitude = Text(
    placeholder="Enter the Latitude", description="Latitude:", disabled=False
)

longitude = Text(
    placeholder="Enter the Longitude", description="Longitude:", disabled=False
)

layer_control = LayersControl()

point_list = SelectMultiple(options=[], value=[], description="Points:", disabled=False)

points = []

button = Button(
    description="Run",
    button_style="success",
    disabled=False,
    tooltip="Click 'Run' to start the Osprey Flask App.",
)
button.on_click(handle_button)

start_date = date_widget("Start Date:")
end_date = date_widget("End Date:")

In [ ]:
m.add_layer(p)
m.add_layer(f)
m.add_layer(c)

m += layer_control

m.on_interaction(handle_click)
m.add_control(legend)

display(m)
display(start_date, end_date)
display(latitude, longitude)
display(point_list, button)

In [ ]:
def client():
    flask_app = create_app()

    # Create a test client using the Flask application configured for testing
    return flask_app.test_client()


def full_rvic_test(url, client, valid_input=True):
    # input_params = urlencode(url)
    input_url = f"/osprey/input?{url}"

    input_response = client.get(input_url)
    if valid_input:
        assert input_response.status_code == 202
    else:
        assert input_response.status_code == 400
        return

    status_url = input_response.data.split()[-1].decode("utf-8")
    status_response = client.get(status_url)

    timeout = 1800  # Time to timeout in seconds
    for i in range(timeout):
        if status_response.data != b"Process is still running.":  # Process is completed
            break
        time.sleep(1)
        status_response = client.get(status_url)
    assert b"Process completed." in status_response.data

    output_url = status_response.data.split()[-1].decode("utf-8")
    output_response = client.get(output_url)
    streamflow_path = output_response.headers.get("Location")
    assert requests.get(streamflow_path).status_code == 200

In [ ]:
test_client = client()

In [ ]:
def build_url(start, end, lat, lon):
   
    start_parsed = str(start.time()).split(":")

    start_val = "&run_startdate=" + str(start.date()) + "-" + start_parsed[0]
    end_val = "&stop_date=" + str(end.date())
    lon_val = "&lons=" + lon
    lat_val = "&lats=" + lat
    url = "case_id=sample" + start_val + end_val + lon_val + lat_val

    print(url)

    return url


url = build_url(start_date.value, end_date.value, latitude.value, longitude.value)
full_rvic_test(url, test_client, valid_input=True)